![terrainbento logo](../../../../media/terrainbento_logo.png)


# terrainbento model Basic model with real DEM


# Introduction to boundary conditions in terrainbento.


## Overview
This tutorial shows example usage of the terrainbento boundary handlers. For comprehensive information about all options and defaults, refer to the [documentation](http://terrainbento.readthedocs.io/en/latest/). 

## Prerequisites
This tutorial assumes you have at least skimmed the [terrainbento manuscript](https://www.geosci-model-dev.net/12/1267/2019/) and worked through the [Introduction to terrainbento](http://localhost:8888/notebooks/example_usage/Introduction_to_terrainbento.ipynb) tutorial.

### terrainbento boundary handlers
terrainbento includes five boundary handlers designed to make it easier to treat different model run boundary conditions. Four boundary handlers modify the model grid in order to change the base level the model sees. The final one calculates how changes in precipitation distribution statistics change the value of erodibility by water. Hyperlinks in the list below go to the documentation for each of the boundary condition handlers. 

1. [`CaptureNodeBaselevelHandler`](https://terrainbento.readthedocs.io/en/latest/source/terrainbento.boundary_handlers.capture_node_baselevel_handler.html?highlight=capture%20node) implements external drainage capture. 
2. [`SingleNodeBaselevelHandler`](https://terrainbento.readthedocs.io/en/latest/source/terrainbento.boundary_handlers.single_node_baselevel_handler.html?highlight=SingleNodeBaselevelHandler) modifies the elevation of one model grid node, intended to be the outlet of a modeled watershed. 
3. [`NotCoreNodeBaselevelHandler`](https://terrainbento.readthedocs.io/en/latest/source/terrainbento.boundary_handlers.not_core_node_baselevel_handler.html?highlight=NotCoreNodeBaselevelHandler) either increments all the core nodes, or all the not-core nodes up or down. 
4. [`GenericFuncBaselevelHandler`](https://terrainbento.readthedocs.io/en/latest/source/terrainbento.boundary_handlers.generic_function_baselevel_handler.html?highlight=GenericFuncBaselevelHandler) is a generic boundary condition handler that modifies the model grid based on a user specified function of the model grid and model time. 
5. [`PrecipChanger`](https://terrainbento.readthedocs.io/en/latest/source/terrainbento.boundary_handlers.precip_changer.html?highlight=PrecipChanger) modifies precipitation distribution parameters (in **St** models) or erodibility by water (all other models). 

If you have additional questions related to using the boundary handlers or your research requires additonal tools to handle boundary conditions, please let us know by making an [Issue on GitHub](https://github.com/TerrainBento/terrainbento/issues). 

In the `SingleNodeBaselevelHandler` and the `NotCoreNodeBaselevelHandler`, rate of baselevel fall at a single node or at all not-core model grid nodes can be specified as a constant rate or a time-elevation history. These and other options are described in the documentation. Note that a model instance can have more than one boundary handler at a time.

The swiss-army knife of boundary condition handling is the `GenericFuncBaselevelHandler` so we will focus on it today. 

### Example Usage

To begin, we will import the required python modules. 

In [ ]:
from terrainbento import Basic

import numpy as np
np.random.seed(42)

import matplotlib
from landlab.plot.imshow import imshow_grid
import matplotlib.pyplot as plt
import holoviews as hv
hv.notebook_extension('matplotlib')
#Ignore warnings 
import warnings
warnings.filterwarnings('ignore')

# Figure properties
%matplotlib inline
import panel as pn
pn.pane.Matplotlib.tight = True

### CSDMS Boundary condition

Now let's create an uplift field where some of the core nodes are being uplifted.

For our example we will create a model grid initially at 0 m elevation at all grid nodes, then we will progressively uplift the model core nodes. We will assume a constant spatial and temporal pattern of uplift rates for teh core nodes.  

If you are not familiar with user defined python functions, consider reviewing [this tutorial](https://www.datacamp.com/community/tutorials/functions-python-tutorial#udf). 

Our function will look as follows:

In [ ]:
def dropping_boundary_condition_CSDMS(grid,t):
    M = np.zeros((25,74))
    M[6:18, 1:5] = 1;M[3:7,1:13]=1;M[18:22,1:13]=1;#M[11:14,7:13]=1;M[12:18, 9:13] = 1;
    M[6:12, 15:18] = 1;M[12:18, 23:26] = 1;M[3:7,15:26]=1;M[10:14,15:26]=1;M[18:22,15:26]=1
    M[3:7, 28:39] = 1;M[3:22,28:32]=1;M[3:22,35:39]=1;M[18:22, 28:39] = 1;
    M[3:7, 41:59] = 1;M[3:22,41:45]=1;M[3:22,48:52]=1;M[3:22,55:59]=1;
    M[6:12, 61:64] = 1;M[12:18, 69:72] = 1;M[3:7,61:72]=1;M[10:14,61:72]=1;M[18:22,61:72]=1
    M = np.flipud(M)
    dzdt = -0.001*M.flatten()
    return dzdt


Next we will make a new model that is exactly the same as the other model but uses the new function and a different output file name and a lower  water_erodibility constant (change to 1e-5)

In [ ]:
basic_basic_csdms_params = {
    # create the Clock.
    "clock": {
        "start": 0,
        "step": 1000,
        "stop": 2e5
    },

    # Create the Grid.
    "grid": {
        "RasterModelGrid": [(25, 74), {
            "xy_spacing": 40
        }, {
            "fields": {
                "node": {
                    "topographic__elevation": {
                        "random": [{
                            "where": "CORE_NODE"
                        }],
                        "constant": [{
                            "value": 1000.
                        }]
                    }
                }
            }
        }]
    },

    # Set up Boundary Handlers
    "boundary_handlers": {
        "GenericFuncBaselevelHandler": {
            "modify_core_nodes" : True,
            "function": dropping_boundary_condition_CSDMS
        }
    },

    # Parameters that control output.
    "output_interval": 1e4,
    "save_first_timestep": True,
    "output_prefix": "output/model_basic_output_intro_bc_csdms",
    "fields": ["topographic__elevation"],

    # Parameters that control process and rates.
    "water_erodibility": 0.0001,
    "m_sp": 0.5,
    "n_sp": 1.0,
    "regolith_transport_parameter": 0,
}

Next we create a model instance

In [ ]:
basic_csdms = Basic.from_dict(basic_basic_csdms_params)

Run it, create an xarray dataset of the model output, and convert it to the holoviews format. 

In [ ]:
basic_csdms.run()
ds_csdms = basic_csdms.to_xarray_dataset(time_unit='years', space_unit='meters')
hvds_topo_csdms = hv.Dataset(ds_csdms.topographic__elevation)

Finally we create an image of the topography with a slider bar. 

In [ ]:
%opts Image style(interpolation='bilinear', cmap='viridis') plot[colorbar=True]
topo_csdms = hvds_topo_csdms.to(hv.Image, ['x', 'y'], label='topo_CSDMS')
topo_csdms.opts(fontsize={
    'title': 10, 
    'labels': 10, 
    'xticks': 10, 
    'yticks': 10,       
    'cticks': 10,
})
topo_csdms

Finally we remove the xarray datasets from and use the model function remove_output_netcdfs to remove the files created by running the model. 

In [ ]:
ds_csdms.close()
basic_csdms.remove_output_netcdfs()

## Next Steps

- We recommend you review the [terrainbento manuscript](https://www.geosci-model-dev.net/12/1267/2019/).

- There are three additional introductory tutorials: 

    1) [Introduction terrainbento](Introduction_to_terrainbento.ipynb) 
    
    2) **This Notebook**: [Introduction to boundary conditions in terrainbento](introduction_to_boundary_conditions.ipynb)
    
    3) [Introduction to output writers in terrainbento](introduction_to_output_writers.ipynb). 
    
    
- Five examples of steady state behavior in coupled process models can be found in the following notebooks:

    1) [Basic](../coupled_process_elements/model_basic_steady_solution.ipynb) the simplest landscape evolution model in the terrainbento package.

    2) [BasicVm](../coupled_process_elements/model_basic_var_m_steady_solution.ipynb) which permits the drainage area exponent to change

    3) [BasicCh](../coupled_process_elements/model_basicCh_steady_solution.ipynb) which uses a non-linear hillslope erosion and transport law

    4) [BasicVs](../coupled_process_elements/model_basicVs_steady_solution.ipynb) which uses variable source area hydrology

    5) [BasisRt](../coupled_process_elements/model_basicRt_steady_solution.ipynb) which allows for two lithologies with different K values
    
    6) [RealDEM](../coupled_process_elements/model_basic_realDEM.ipynb) Run the basic terrainbento model with a real DEM as initial condition. 